In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module
from datetime import datetime

#ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')
ws = Workspace.get(name='itp-pilot', subscription_id='4aaa645c-5ae2-4ae9-a17a-84b9023bc56a', resource_group='itp-pilot-ResGrp')

modulefunc = Module.from_yaml(ws, yaml_file='./noop/1in2out.spec.yaml')
data = Dataset.get_by_name(ws, 'training_data')

In [2]:
@dsl.pipeline(
    name='A huge pipeline composed with nodes 1 in 2 outs',
    description='A sample',
    default_compute_target='k80-16-b'
)
def cell_division():
    layer = 11
    nodes = []
    nodes.append(modulefunc(input1=data))
    for i in range(0, layer-1):
        print('i=', i, ' nodes len=', len(nodes))
        current_layer_nodes = []
        for j in range(0, pow(2,i)):
            print(datetime.now().strftime("%d/%m/%Y %H:%M:%S"), '\tj=', j)
            n = nodes[-j-1]
            current_layer_nodes.append(modulefunc(input1=n.outputs.output1))
            current_layer_nodes.append(modulefunc(input1=n.outputs.output2))
        nodes = nodes + current_layer_nodes
    
    return {**nodes[-1].outputs}

pipeline = cell_division()
#pipeline.validate()

run = pipeline.submit(experiment_name='samples')


i= 0  nodes len= 1
16/06/2020 10:45:25 	j= 0
i= 1  nodes len= 3
16/06/2020 10:45:25 	j= 0
16/06/2020 10:45:25 	j= 1
i= 2  nodes len= 7
16/06/2020 10:45:25 	j= 0
16/06/2020 10:45:25 	j= 1
16/06/2020 10:45:25 	j= 2
16/06/2020 10:45:25 	j= 3
i= 3  nodes len= 15
16/06/2020 10:45:25 	j= 0
16/06/2020 10:45:25 	j= 1
16/06/2020 10:45:25 	j= 2
16/06/2020 10:45:25 	j= 3
16/06/2020 10:45:25 	j= 4
16/06/2020 10:45:25 	j= 5
16/06/2020 10:45:25 	j= 6
16/06/2020 10:45:25 	j= 7
i= 4  nodes len= 31
16/06/2020 10:45:25 	j= 0
16/06/2020 10:45:25 	j= 1
16/06/2020 10:45:25 	j= 2
16/06/2020 10:45:25 	j= 3
16/06/2020 10:45:25 	j= 4
16/06/2020 10:45:25 	j= 5
16/06/2020 10:45:25 	j= 6
16/06/2020 10:45:25 	j= 7
16/06/2020 10:45:25 	j= 8
16/06/2020 10:45:25 	j= 9
16/06/2020 10:45:25 	j= 10
16/06/2020 10:45:25 	j= 11
16/06/2020 10:45:25 	j= 12
16/06/2020 10:45:25 	j= 13
16/06/2020 10:45:25 	j= 14
16/06/2020 10:45:25 	j= 15
i= 5  nodes len= 63
16/06/2020 10:45:25 	j= 0
16/06/2020 10:45:25 	j= 1
16/06/2020 10:45:25

In [ ]:
run.wait_for_completion(show_output=True)